# Load candidates

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
CACHE = '/home/anhphantq/vdb/cache_train'
import os
import pickle
from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
import psutil

train_data = []

with open(CACHE + '/metadata', 'rb') as f:
  metadata = pickle.load(f)

num_frags = min(metadata['num_frags'], 10000)
max_session = metadata['max_session']

with tqdm(total= int (psutil.virtual_memory().total  / 1024  / 1024 / 1024), desc='RAM (GB)', position=0, leave= False) as rambar:

  for i in tqdm(range(0, num_frags)):

    train_data.append(pd.read_parquet(CACHE + f'/data_{i}.parquet'))
    rambar.n= int(psutil.virtual_memory().used / 1024  / 1024 / 1024)
    rambar.refresh()




RAM (GB):   0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/1801 [00:00<?, ?it/s]

In [3]:
train_data = pd.concat(train_data)

In [4]:
candidates = train_data
candidates.shape

(98120900, 138)

In [5]:
import pandas as pd
pd.set_option('display.max_columns', candidates.shape[1])

In [6]:
candidates

,user,item,num_clicks,num_carts,num_orders,item_degree,item_pr,recent_num_clicks,recent_num_carts,recent_num_orders,item_recent_degree,item_recent_pr,recent_day7_type0,recent_day7_type1,recent_day7_type2,recent_day6_type0,recent_day6_type1,recent_day6_type2,recent_day5_type0,recent_day5_type1,recent_day5_type2,recent_day4_type0,recent_day4_type1,recent_day4_type2,recent_day3_type0,recent_day3_type1,recent_day3_type2,recent_day2_type0,recent_day2_type1,recent_day2_type2,recent_day1_type0,recent_day1_type1,recent_day1_type2,item_glob_last_action,item_glob_first_action,item_glob_time_decay_sum,item_glob_time_decay_sum_click,item_glob_time_decay_sum_cart,item_glob_time_decay_sum_order,item_glob_item_glob_durability,qou_num_cosub_mean,qou_coclick_sub_coef_mean,qou_cocart_sub_coef_mean,qou_coorder_sub_coef_mean,qou_coclick_time_decay_mean,qou_cocart_time_decay_mean,qou_coorder_time_decay_mean,qou_num_appearance_mean,qou_num_in_k_most_recent_items_mean,qou_num_happend_later_mean,qou_num_happend_before_mean,qou_happend_later_ratio_mean,qou_last_interact_mean,qou_lincom_sub_coef_1_3_10_mean,qou_lincom_time_decay_1_3_10_mean,qou_lincom_sub_coef_1_10_3_mean,qou_lincom_time_decay_1_10_3_mean,qou_lincom_sub_coef_3_1_10_mean,qou_lincom_time_decay_3_1_10_mean,qou_lincom_sub_coef_3_10_1_mean,qou_lincom_time_decay_3_10_1_mean,qou_lincom_sub_coef_10_1_3_mean,qou_lincom_time_decay_10_1_3_mean,qou_lincom_sub_coef_10_3_1_mean,qou_lincom_time_decay_10_3_1_mean,qou_lincom_sub_coef_0.5_10_0.5_mean,qou_lincom_time_decay_0.5_10_0.5_mean,qou_lincom_sub_coef_10_0.5_0.5_mean,qou_lincom_time_decay_10_0.5_0.5_mean,qou_lincom_sub_coef_0.5_0.5_10_mean,qou_lincom_time_decay_0.5_0.5_10_mean,qou_num_cosub_sqrt_num_cousers,qou_coclick_sub_coef_sqrt_num_cousers,qou_cocart_sub_coef_sqrt_num_cousers,qou_coorder_sub_coef_sqrt_num_cousers,qou_coclick_time_decay_sqrt_num_cousers,qou_cocart_time_decay_sqrt_num_cousers,qou_coorder_time_decay_sqrt_num_cousers,qou_num_appearance_sqrt_num_cousers,qou_num_in_k_most_recent_items_sqrt_num_cousers,qou_num_happend_later_sqrt_num_cousers,qou_num_happend_before_sqrt_num_cousers,qou_happend_later_ratio_sqrt_num_cousers,qou_last_interact_sqrt_num_cousers,num_sub,consistency,num_actions,user_degree,user_pr,user_recent_degree,user_recent_pr,cointeractions_coef_clicks_clicks,cointeractions_coef_clicks_carts,cointeractions_coef_clicks_orders,cointeractions_coef_carts_clicks,cointeractions_coef_carts_carts,cointeractions_coef_carts_orders,cointeractions_coef_orders_clicks,cointeractions_coef_orders_carts,cointeractions_coef_orders_orders,cointeractions_time_decay_clicks_clicks,cointeractions_time_decay_clicks_carts,cointeractions_time_decay_clicks_orders,cointeractions_time_decay_carts_clicks,cointeractions_time_decay_carts_carts,cointeractions_time_decay_carts_orders,cointeractions_time_decay_orders_clicks,cointeractions_time_decay_orders_carts,cointeractions_time_decay_orders_orders,fitness,inter_clicks,inter_carts,inter_orders,inter_num_sub,inter_time_decay,inter_lts,inter_fts,inter_durability,inter_num_interacts,is_level1,user_clicks_cnt,user_carts_cnt,user_orders_cnt,user_total_actions_cnt,user_clicks_ratio,user_carts_ratio,user_orders_ratio,user_ss_ts_max,user_ss_ts_min,user_ss_ts_mean,item_item_cnt,item_item_clicks_cnt,item_item_carts_cnt,item_item_orders_cnt,item_item_total_actions_cnt,item_item_clicks_ratio,item_item_carts_ratio,item_item_orders_ratio
0,11612939.0,12965372.0,1598.0,102.0,39.0,943.0,4.855294e-06,658.0,54.0,21.0,305.0,3.577215e-06,88.0,10.0,4.0,58.0,6.0,3.0,60.0,3.0,0.0,87.0,7.0,2.0,103.0,4.0,0.0,80.0,9.0,3.0,60.0,4.0,2.0,20.986620,0.007025,721.630615,659.347656,45.976597,16.306309,20.979595,1.383855,1.315631,0.124667,0.000000,9.441611e-01,8.667468e-02,0.000000e+00,1.310542,0.215630,1.838246,1.381774,0.817349,1.033126,1.196428,8.539671e-01,1.066915,7.622917e-01,1.292174,9.257514e-01,1.221740,8.755513e-01,1.311383,9.409398e-01,1.306242,9.372653e-01,0.913321,6.515723e-01,1.313918,9.428899e-01,1.211587,8.615743e-01,0

# Train params

In [7]:
from meow_utils import *

import numba as nb
from numba import prange
@nb.njit(nb.float64(nb.float64[:], nb.float32[:], nb.int64[:]))
def recall20(approxes, target, groups):
    total = 0
    nonempty = 0
    group_ends = np.concatenate((np.ravel(np.argwhere(groups[:-1] != groups[1:])), np.array([len(groups) -1]))) + 1
    group_starts = np.concatenate((np.array([0]), group_ends[:-1]))

    for i in prange(len(group_starts)):
        group_end = group_ends[i]
        group_start = group_starts[i]
        ranks = np.argsort(approxes[group_start:group_end])[::-1]
        hits = 0
        for i in prange(min(len(ranks), 20)):
            hits += target[group_start + ranks[i]]

        actual = min(20, target[group_start:group_end].sum())
        if actual > 0:
            total += hits / actual
            nonempty += 1
    return total / nonempty

class GlobalRecall20(object):
    def get_final_error(self, error, weight):
        return error

    def is_max_optimal(self):
        return True

    def evaluate(self, approxes, target, weight = None):
        try:
            groups = weight.astype(int)
        except:
            return 0, 0
            
        
        
        return recall20(approxes[0], target, groups), 0
  

PARAMS = {
    # 'learning_rate': 0.5,
    'iterations': 1000,
    # 'iterations' : 1,
    'early_stopping_rounds' : 20,
    'max_bin': 256,
    'depth': 7,
    'eval_metric': GlobalRecall20(),
    'use_best_model': True,
    'thread_count': -1,
    'subsample': 1
}
N_SPLITS = 5
NUM_CLICK_SAMPLES = 1
NUM_SAMPLES = 3
MODEL_PATH = '/home/anhphantq/meowmeow'
VERSION = '14'


# Train for each type

In [8]:
from meow_utils import get_len_group

In [9]:
import pandas as pd 
import numpy as np
tar = pd.read_parquet('/home/anhphantq/otto/splitted_data/test_labels.parquet')
tar = tar.loc[(tar['session'] <= candidates['user'].max()) & (tar['session'] >= candidates['user'].min()) ]
aids = tar.ground_truth.explode().astype('int32').rename('item')
tar = tar[['session', 'type']].rename({'session':'user'},axis=1)
tar = tar.merge(aids, left_index=True, right_index=True, how='left')
tar['type'] = tar['type'].apply(lambda x: 0 if x == 'clicks' else 1 if x == 'carts' else 2)
tar['label'] = 1
tar['item'] = tar['item'] + max_session
tar['user_type'] = tar['user'] * 10 + tar['type']

In [10]:
labels = []
users = candidates.user.values
items = candidates.item.values
from tqdm.notebook import tqdm
for i in tqdm(range(3)):
  cnt = 0
  tar_small = tar[tar['type'] == i].sort_values(['user', 'item']).set_index(['user', 'item'])
  
  tar_small_index = set(tar_small.index.tolist())
  label = []

  for i in tqdm(range(candidates.shape[0]), leave = False):
    if (users[i], items[i]) in tar_small_index:
      label.append(1)
      cnt += 1
    else: 
      label.append(0)
  labels.append(label)
  print(f'Found {cnt} positives', end = ' ')
del tar, tar_small_index
  



  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 753055 positives 

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 192092 positives 

  0%|          | 0/98120900 [00:00<?, ?it/s]

Found 144328 positives 

In [11]:
labels = np.array(labels)

In [12]:
from sklearn.model_selection import GroupKFold
import os
import matplotlib.pyplot as plt
import shutil
from meow_utils import plot_importance
from catboost import CatBoostRanker, Pool
import seaborn as sns
sns.set()
%matplotlib inline
def train(t, id_type, draw_importance = False):
  global candidates, labels
  name = f'/{t}_only_'
  try:
    shutil.rmtree(MODEL_PATH + name + VERSION)
  except:
    pass
  else:
    print('remove old version')
  fig, axs = plt.subplots(1, N_SPLITS, figsize = (300, 30))
  skf = GroupKFold(n_splits=N_SPLITS)
  
  
  X_all = candidates.iloc[:, 2 :]
  y_all = labels[id_type]
  
  
  print('Building pool...')
  data_all = Pool(X_all, y_all, thread_count = os.cpu_count())
  print('Pool builed!!!!!!')
  for fold,(train_idx, valid_idx) in enumerate(skf.split(candidates, labels[id_type], groups=candidates['user'] )):
      neg_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 0))
      pos_index = np.ravel(np.argwhere(labels[id_type][train_idx] == 1))
      #downsample many times
      for sample in range(NUM_SAMPLES if id_type != 0 else NUM_CLICK_SAMPLES):
        print(f'FOLD: {fold} Sample: {sample}')
        n_index = np.random.permutation(neg_index)[:len(pos_index) * 20]
        index = np.union1d(n_index, pos_index)
        
        data_train = data_all.slice(train_idx).slice(index)
        data_train.set_group_id(candidates['user'].values.astype(int)[train_idx][index])
        data_valid = data_all.slice(valid_idx)
        
        # print(len(candidates.user.values[valid_idx]))
        data_valid.set_group_id(candidates['user'].values.astype(int)[valid_idx])
        data_valid.set_group_weight(candidates.user.values[valid_idx])

        params = {'thread_count' : os.cpu_count(),
        **PARAMS}
        model = CatBoostRanker(**params)
        
        model.fit(data_train, eval_set = data_valid)
        
        try:
          os.mkdir(MODEL_PATH + name + VERSION)
        except:
          pass
        if fold < 5 and sample == 0 and draw_importance:
          plot_importance(X_all.columns.tolist(), model.get_feature_importance(data = data_valid, type = 'LossFunctionChange'), ax = axs[fold])
          axs[fold].bar_label(axs[fold].containers[0])
        model.save_model(MODEL_PATH + name + VERSION + f'/meowmewo_fold{fold}_sample{sample}')
  


In [13]:
for id_type, t in enumerate(['clicks', 'carts', 'orders']):
  train(t, id_type, draw_importance = False)

Building pool...
Pool builed!!!!!!
FOLD: 0 Sample: 0


/home/anhphantq/.local/lib/python3.8/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'astype' of type none

File "../../../tmp/ipykernel_2998416/94278090.py", line 35:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_2998416/94278090.py (35)

File "../../../tmp/ipykernel_2998416/94278090.py", line 35:
<source missing, REPL/exec in use?>

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


0:	learn: 0.0000000	test: 0.8596931	best: 0.8596931 (0)	total: 4.61s	remaining: 1h 16m 44s
1:	learn: 0.0000000	test: 0.8635024	best: 0.8635024 (1)	total: 7.82s	remaining: 1h 5m
2:	learn: 0.0000000	test: 0.8649425	best: 0.8649425 (2)	total: 10.9s	remaining: 1h 7s
3:	learn: 0.0000000	test: 0.8665021	best: 0.8665021 (3)	total: 14.1s	remaining: 58m 24s
4:	learn: 0.0000000	test: 0.8668538	best: 0.8668538 (4)	total: 17.3s	remaining: 57m 14s
5:	learn: 0.0000000	test: 0.8670662	best: 0.8670662 (5)	total: 20.4s	remaining: 56m 21s
6:	learn: 0.0000000	test: 0.8669335	best: 0.8670662 (5)	total: 23.5s	remaining: 55m 28s
7:	learn: 0.0000000	test: 0.8671193	best: 0.8671193 (7)	total: 26.8s	remaining: 55m 17s
8:	learn: 0.0000000	test: 0.8675042	best: 0.8675042 (8)	total: 29.9s	remaining: 54m 54s
9:	learn: 0.0000000	test: 0.8675241	best: 0.8675241 (9)	total: 33s	remaining: 54m 27s
10:	learn: 0.0000000	test: 0.8675241	best: 0.8675241 (9)	total: 36.2s	remaining: 54m 16s
11:	learn: 0.0000000	test: 0.86747

/home/anhphantq/.local/lib/python3.8/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'astype' of type none

File "../../../tmp/ipykernel_2998416/94278090.py", line 35:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_2998416/94278090.py (35)

File "../../../tmp/ipykernel_2998416/94278090.py", line 35:
<source missing, REPL/exec in use?>

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


0:	learn: 0.0000000	test: 0.8597881	best: 0.8597881 (0)	total: 3.33s	remaining: 55m 23s
1:	learn: 0.0000000	test: 0.8651442	best: 0.8651442 (1)	total: 6.41s	remaining: 53m 20s
2:	learn: 0.0000000	test: 0.8655756	best: 0.8655756 (2)	total: 9.68s	remaining: 53m 38s
3:	learn: 0.0000000	test: 0.8663322	best: 0.8663322 (3)	total: 13.4s	remaining: 55m 24s
4:	learn: 0.0000000	test: 0.8664583	best: 0.8664583 (4)	total: 17.5s	remaining: 58m 8s
5:	learn: 0.0000000	test: 0.8667569	best: 0.8667569 (5)	total: 21.3s	remaining: 58m 43s
6:	learn: 0.0000000	test: 0.8671684	best: 0.8671684 (6)	total: 25.2s	remaining: 59m 36s
7:	learn: 0.0000000	test: 0.8672282	best: 0.8672282 (7)	total: 29.2s	remaining: 1h 22s
8:	learn: 0.0000000	test: 0.8671618	best: 0.8672282 (7)	total: 33.1s	remaining: 1h 39s
9:	learn: 0.0000000	test: 0.8673874	best: 0.8673874 (9)	total: 36.9s	remaining: 1h 53s
10:	learn: 0.0000000	test: 0.8677060	best: 0.8677060 (10)	total: 40.1s	remaining: 1h 4s
11:	learn: 0.0000000	test: 0.8677392

...

0:	learn: 0.0000000	test: 0.9426264	best: 0.9426264 (0)	total: 2.03s	remaining: 33m 44s
1:	learn: 0.0000000	test: 0.9479383	best: 0.9479383 (1)	total: 4.08s	remaining: 33m 55s
2:	learn: 0.0000000	test: 0.9502691	best: 0.9502691 (2)	total: 6.33s	remaining: 35m 3s
3:	learn: 0.0000000	test: 0.9516029	best: 0.9516029 (3)	total: 8.48s	remaining: 35m 12s
4:	learn: 0.0000000	test: 0.9517778	best: 0.9517778 (4)	total: 10.6s	remaining: 35m 13s
5:	learn: 0.0000000	test: 0.9525730	best: 0.9525730 (5)	total: 13s	remaining: 35m 49s
6:	learn: 0.0000000	test: 0.9525753	best: 0.9525753 (6)	total: 15.4s	remaining: 36m 19s
7:	learn: 0.0000000	test: 0.9527852	best: 0.9527852 (7)	total: 17.8s	remaining: 36m 41s
8:	learn: 0.0000000	test: 0.9525355	best: 0.9527852 (7)	total: 20.1s	remaining: 36m 56s
9:	learn: 0.0000000	test: 0.9522796	best: 0.9527852 (7)	total: 22.5s	remaining: 37m 8s
10:	learn: 0.0000000	test: 0.9525548	best: 0.9527852 (7)	total: 24.9s	remaining: 37m 17s
11:	learn: 0.0000000	test: 0.952753

/home/anhphantq/.local/lib/python3.8/site-packages/catboost/core.py:1759: UserWarning: Failed to optimize method "evaluate" in the passed object:
Failed in nopython mode pipeline (step: nopython frontend)
Unknown attribute 'astype' of type none

File "../../../tmp/ipykernel_2998416/94278090.py", line 35:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_2998416/94278090.py (35)

File "../../../tmp/ipykernel_2998416/94278090.py", line 35:
<source missing, REPL/exec in use?>

  self._object._train(train_pool, test_pool, params, allow_clear_pool, init_model._object if init_model else None)


0:	learn: 0.0000000	test: 0.9441541	best: 0.9441541 (0)	total: 1.84s	remaining: 30m 34s
1:	learn: 0.0000000	test: 0.9475220	best: 0.9475220 (1)	total: 3.65s	remaining: 30m 19s
2:	learn: 0.0000000	test: 0.9506619	best: 0.9506619 (2)	total: 5.86s	remaining: 32m 26s
3:	learn: 0.0000000	test: 0.9522351	best: 0.9522351 (3)	total: 7.98s	remaining: 33m 7s
4:	learn: 0.0000000	test: 0.9519573	best: 0.9522351 (3)	total: 10.3s	remaining: 34m 11s
5:	learn: 0.0000000	test: 0.9518035	best: 0.9522351 (3)	total: 12.5s	remaining: 34m 25s
6:	learn: 0.0000000	test: 0.9523726	best: 0.9523726 (6)	total: 14.7s	remaining: 34m 42s
7:	learn: 0.0000000	test: 0.9529527	best: 0.9529527 (7)	total: 16.9s	remaining: 34m 54s
8:	learn: 0.0000000	test: 0.9529192	best: 0.9529527 (7)	total: 19.1s	remaining: 35m 4s
9:	learn: 0.0000000	test: 0.9525838	best: 0.9529527 (7)	total: 21.5s	remaining: 35m 24s
10:	learn: 0.0000000	test: 0.9527466	best: 0.9529527 (7)	total: 23.9s	remaining: 35m 44s
11:	learn: 0.0000000	test: 0.9523

: 

: 